In [6]:
!pip install pandas

In [7]:
import sys
print(sys.executable)



d:\chicago\.venv\Scripts\python.exe


In [8]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import gc
import logging

In [9]:
import pymysql

# Establishing the connection
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root"
)
print(connection)
# Creating a cursor object
mycursor= connection.cursor()
print(mycursor)

In [14]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_table_and_insert(csv_file_path, table_name, host, user, password, database, batch_size=100):
    try:
        # Connect to MySQL
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        mycursor = connection.cursor()

        # Table structure based on the provided columns
        create_table_query = """
        CREATE TABLE IF NOT EXISTS `chicago` (
            `ID` INT,
            `Case Number` VARCHAR(50),
            `Date` DATETIME,
            `Block` VARCHAR(255),
            `IUCR` VARCHAR(10),
            `Primary Type` VARCHAR(100),
            `Description` VARCHAR(255),
            `Location Description` VARCHAR(255),
            `Arrest` BOOLEAN,
            `Domestic` BOOLEAN,
            `Beat` INT,
            `District` INT,
            `FBI Code` VARCHAR(10),
            `Year` INT,
            `Updated On` DATETIME,
            `Latitude` FLOAT,
            `Longitude` FLOAT
        );
        """
        mycursor.execute(create_table_query)
        logging.info(f"Table `{table_name}` created successfully!")

        # Process CSV in chunks to avoid memory issues
        total_rows_inserted = 0
        for chunk in pd.read_csv(csv_file_path, chunksize=batch_size):
            placeholders = ', '.join(['%s'] * len(chunk.columns))
            insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
            mycursor.executemany(insert_query, chunk.values.tolist())
            connection.commit()
            total_rows_inserted += mycursor.rowcount
            logging.info(f"Inserted {mycursor.rowcount} rows.")

            # Clear memory after processing each chunk
            del chunk
            gc.collect()  # Run garbage collection to free up memory

        logging.info(f"Total rows inserted: {total_rows_inserted}")
        

    except Error as e:
        logging.error(f"Error: {e}")
    except Exception as ex:
        logging.error(f"Unexpected error: {ex}")
    finally:
        if 'connection' in locals():
            connection.close()
            logging.info("MySQL connection closed.")

# Example Usage
create_table_and_insert(
    csv_file_path=r"C:\Users\Digital Suppliers\Downloads\crime_cleaned.csv",  # Path to your CSV file
    table_name="chicago",                                        # Table name
    host="localhost",                                            # MySQL host
    user="root",                                                 # MySQL username
    password="root",                                    # MySQL password
    database="crime",                                            # MySQL database
    batch_size=1000000                                              # Number of rows per batch
)

2025-01-16 17:36:20,301 - INFO - Table `chicago` created successfully!
2025-01-16 17:36:51,944 - INFO - Inserted 549999 rows.
2025-01-16 17:36:52,434 - INFO - Total rows inserted: 549999
2025-01-16 17:36:52,437 - INFO - MySQL connection closed.
